In [1]:
import pandas as pd

In [2]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

In [3]:
import seaborn as sns

In [4]:
## create dict sites plots 

merged_hapFIRE_allele_frequency = pd.read_csv('../key_files/merged_hapFIRE_allele_frequency.txt',nrows=1, sep = '\t')

merged_hapFIRE_allele_frequency = merged_hapFIRE_allele_frequency.T

merged_hapFIRE_allele_frequency = merged_hapFIRE_allele_frequency.reset_index()

merged_hapFIRE_allele_frequency['site'] = merged_hapFIRE_allele_frequency['index'].str.split('_').str[0]
merged_hapFIRE_allele_frequency['gen'] = merged_hapFIRE_allele_frequency['index'].str.split('_').str[1]
merged_hapFIRE_allele_frequency['plot'] = merged_hapFIRE_allele_frequency['index'].str.split('_').str[2]

samples = merged_hapFIRE_allele_frequency.drop([0, 'index'],axis=1).drop_duplicates()

samples_at_least_2_years = samples[['site', 'plot']].value_counts()[samples[['site', 'plot']].value_counts() > 1 ].reset_index()

sites_plots_w_at_least_2_years =  samples_at_least_2_years[['site', 'plot']].groupby('site')['plot'].unique().to_dict()

In [5]:
## test 

In [6]:
flowers = pd.read_csv('../key_files/merged_sample_table.csv')

var_pos = pd.read_csv('../key_files/var_pos_grenenet.csv')
#path_meixi = '/carnegie/data/Shared/Labs/Moi/Everyone/meixilin'

num_flowers_map = flowers.set_index('sample_name')['total_flower_counts'].to_dict()

merged_hapFIRE_allele_frequency = pd.read_csv('../key_files/merged_hapFIRE_allele_frequency.txt',nrows=1, sep = '\t')

merged_hapFIRE_allele_frequency = merged_hapFIRE_allele_frequency.T

merged_hapFIRE_allele_frequency = merged_hapFIRE_allele_frequency.reset_index()

merged_hapFIRE_allele_frequency['site'] = merged_hapFIRE_allele_frequency['index'].str.split('_').str[0]
merged_hapFIRE_allele_frequency['gen'] = merged_hapFIRE_allele_frequency['index'].str.split('_').str[1]
merged_hapFIRE_allele_frequency['plot'] = merged_hapFIRE_allele_frequency['index'].str.split('_').str[2]


#mask = var_pos['maf05filter'].notna()

In [7]:
site = '1'

In [8]:
site_plot_samples = [i for i in merged_hapFIRE_allele_frequency['index'] if i.startswith(str(site) + '_')]

In [9]:
merged_hapFIRE_allele_frequency = pd.read_csv('../key_files/merged_hapFIRE_allele_frequency.txt',nrows=500, sep = '\t', usecols = site_plot_samples)

#merged_hapFIRE_allele_frequency = merged_hapFIRE_allele_frequency[mask]

merged_hapFIRE_allele_frequency = merged_hapFIRE_allele_frequency.reset_index(drop=True)



## generate the allale counts
num_flowers_map = flowers.set_index('sample_name')['total_flower_counts'].to_dict()

allele_counts = {}
for i in merged_hapFIRE_allele_frequency.columns:
    num_flowers = num_flowers_map[i]
    allele_counts[i] = merged_hapFIRE_allele_frequency[i] * num_flowers * 2

allele_counts = pd.concat(allele_counts,axis=1)

allele_counts = allele_counts.T

allele_counts = allele_counts.reset_index()


allele_counts['gen'] = allele_counts['index'].str.split('_').str[1].astype(int)
allele_counts['plot'] = allele_counts['index'].str.split('_').str[2].astype(int)


allele_counts = allele_counts.drop('index',axis=1)

allele_counts = allele_counts.melt(id_vars=['gen', 'plot'])

allele_counts.columns = ['gen', 'plot', 'snp', 'count']

allele_counts = allele_counts.drop('plot',axis=1)

In [10]:
allele_counts

,gen,snp,count
0,1,0,13.293954
1,1,0,8.172854
2,1,0,20.999298
3,1,0,24.278624
4,1,0,24.129688
...,...,...,...
16995,3,499,0.404056
16996,3,499,1.897332
16997,3,499,0.017204
16998,3,499,28.722642


In [11]:
merged_hapFIRE_allele_frequency = pd.read_csv('../key_files/merged_hapFIRE_allele_frequency.txt',nrows=1, sep = '\t')

merged_hapFIRE_allele_frequency = merged_hapFIRE_allele_frequency.T

merged_hapFIRE_allele_frequency = merged_hapFIRE_allele_frequency.reset_index()

merged_hapFIRE_allele_frequency['site'] = merged_hapFIRE_allele_frequency['index'].str.split('_').str[0]
merged_hapFIRE_allele_frequency['gen'] = merged_hapFIRE_allele_frequency['index'].str.split('_').str[1]
merged_hapFIRE_allele_frequency['plot'] = merged_hapFIRE_allele_frequency['index'].str.split('_').str[2]

In [12]:
unique_sites = merged_hapFIRE_allele_frequency['site'].unique()

In [13]:
pwd -P

'/carnegie/nobackup/scratch/tbellagio/gea_grene-net/snp_origin'

In [14]:
import random
import subprocess

In [59]:
# create sbatch files to submit on cedar server
shfiles = []
for site in sites_plots_w_at_least_2_years:
    seed = random.randint(1,100000000)
    file = f'changep_time{site}.sh'
    cmd = f'python calc_delta_time_allplots_binreg.py {site}'
    text = f'''#!/bin/bash
#SBATCH --job-name=changep_time_{site}
#SBATCH --time=24:00:00  # 
#SBATCH --ntasks=1
#SBATCH --mem-per-cpu=30gb
#SBATCH --output=changep_time_{site}_%j.out
#SBATCH --error=changep_time_{site}_%j.err
#SBATCH --mail-user=tbellagio@carnegiescience.edu
#SBATCH --mail-type=FAIL

source /home/tbellagio/miniforge3/etc/profile.d/conda.sh
conda activate /home/tbellagio/miniforge3/envs/pipeline_snakemake
cd /carnegie/nobackup/scratch/tbellagio/gea_grene-net/snp_origin
{cmd}

'''
    with open(file, 'w') as o:
        o.write("%s" % text)
    shfiles.append(file)

In [60]:
subprocess.run(["sbatch", shfiles[0]], check=True)

Submitted batch job 31624


CompletedProcess(args=['sbatch', 'changep_time1.sh'], returncode=0)

In [61]:
## now run the shfiles
for shfile in shfiles[1:]:
    # Submit each sbatch script to the SLURM scheduler
    subprocess.run(["sbatch", shfile], check=True)

Submitted batch job 31626
Submitted batch job 31627
Submitted batch job 31628
Submitted batch job 31629
Submitted batch job 31630
Submitted batch job 31631
Submitted batch job 31632
Submitted batch job 31633
Submitted batch job 31634
Submitted batch job 31635
Submitted batch job 31636
Submitted batch job 31637
Submitted batch job 31638
Submitted batch job 31639
Submitted batch job 31640
Submitted batch job 31641
Submitted batch job 31642
Submitted batch job 31643
Submitted batch job 31644
Submitted batch job 31645
Submitted batch job 31646
Submitted batch job 31647
Submitted batch job 31648
Submitted batch job 31649
Submitted batch job 31650
